<a href="https://colab.research.google.com/github/priyanshgupta1998/All_codes/blob/master/acv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Preliminaries
import numpy as np 
import matplotlib.pyplot as plt

import pandas as pd
import urllib
import csv

In [0]:
from sklearn import datasets, preprocessing, feature_extraction

In [0]:
from sklearn import linear_model, svm, metrics, ensemble, tree
from sklearn.decomposition import PCA

In [0]:
# import gzip
# with gzip.open('/home/kddcup.data_10_percent.gz') as f:
#     train_data = pd.read_csv(f)
# train_data.shape

# with gzip.open('/home/corrected.gz') as f:
#     test_data =  pd.read_csv(f)
# test_data.shape

In [0]:
names=['Duration', 'protocol_type', 'Service', 'Flag', 'src_bytes', 'dst_bytes', 'Land', 'wrong_fragment', 'Urgent', 'Hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'Count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','Class']

In [0]:
train_multiclass = pd.read_csv('/home/kddcup.data_10_percent_corrected', quotechar=',', skipinitialspace=True , names=names)

In [118]:
train_multiclass['Class'].unique()

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

In [65]:
train_multiclass.shape

(494021, 42)

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='smurf.')|(train_multiclass['Class'] =='neptune.') | (train_multiclass['Class'] =='back.') | (train_multiclass['Class'] =='teardrop.') |(train_multiclass['Class'] =='pod.')| (train_multiclass['Class']=='land.'),'Class'] = 'DOS_ATTACK'

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='satan.')|(train_multiclass['Class'] =='ipsweep.') | (train_multiclass['Class'] =='portsweep.') | (train_multiclass['Class'] =='nmap.'),'Class'] = 'probe'

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='spy.')|(train_multiclass['Class'] =='phf.')|(train_multiclass['Class'] =='multihop.')|(train_multiclass['Class'] =='ftp_write.') | (train_multiclass['Class'] =='imap.') | (train_multiclass['Class'] =='warezmaster.') |(train_multiclass['Class'] =='guess_passwd.')| (train_multiclass['Class']=='warezclient.'),'Class'] = 'r2l'

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='buffer_overflow.')|(train_multiclass['Class'] =='rootkit.') | (train_multiclass['Class'] =='loadmodule.') | (train_multiclass['Class'] =='perl.'),'Class']='u2r'

In [0]:

train_multiclass.loc[(train_multiclass['Class'] =='normal.'),'Class'] = 'normal'

In [71]:
attr_encoder = feature_extraction.DictVectorizer(sparse=False)
attr_encoder

DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [72]:
label_encoder = preprocessing.LabelEncoder()
label_encoder

LabelEncoder()

In [73]:
train_data_df_m = attr_encoder.fit_transform(train_multiclass.iloc[:,:-1].T.to_dict().values())
np.shape(train_data_df_m)

(494021, 118)

In [74]:
train_target_df_m= label_encoder.fit_transform(train_multiclass.iloc[:,-1])  #prediction purose __>target_data
np.shape(train_target_df_m)

(494021,)

In [0]:
train_data_decoded_m = pd.DataFrame(train_data_df_m)
train_target_decoded_m = pd.DataFrame(train_target_df_m)

In [76]:
print(np.shape(train_data_decoded_m))
print(np.shape(train_target_decoded_m))

(494021, 118)
(494021, 1)


In [0]:
#load some modules to help
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [78]:
train_data_pca2 = PCA(n_components=29).fit_transform(train_data_decoded_m)
np.shape(train_data_pca2)

(494021, 29)

In [0]:
train_data_pca_df2 = pd.DataFrame(train_data_pca2)

In [0]:
#Creating our scaler and applyting it to our dataset after feature reduction
standard_scaler = preprocessing.StandardScaler()

In [81]:
train_ratio_standard_scaled_values2 = standard_scaler.fit_transform(train_data_pca_df2.values)
np.shape(train_ratio_standard_scaled_values2)

(494021, 29)

In [0]:
train_data_scaled2=pd.DataFrame(train_ratio_standard_scaled_values2)

In [83]:
clf = svm.SVC(kernel='linear',class_weight="balanced", max_iter=100000000)
clf.fit(train_data_scaled2, train_target_decoded_m[0])

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=100000000, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [84]:
train_multiclass['Class'].unique()

array(['normal', 'u2r', 'DOS_ATTACK', 'r2l', 'probe'], dtype=object)

#Test Files

In [97]:
test_multiclass = pd.read_csv('/home/test.csv', quotechar=',', skipinitialspace=True, names=names)
test_multiclass.shape

(29, 42)

In [0]:
test_data_df_m = attr_encoder.transform(test_multiclass.iloc[:,:-1].T.to_dict().values())

In [99]:
np.shape(test_data_df_m)

(29, 118)

In [109]:
test_target_df_m = label_encoder.transform(test_multiclass.iloc[:,-1])  #takes the target from the test dataset
np.shape(test_target_df_m )

(29,)

In [0]:
test_data_decoded_m = pd.DataFrame(test_data_df_m)
test_target_decoded_m = pd.DataFrame(test_target_df_m)

In [111]:
print(np.shape(test_data_decoded_m))
print(np.shape(test_target_decoded_m))

(29, 118)
(29, 1)


In [103]:
test_data_pca2 = PCA(n_components=29).fit_transform(test_data_decoded_m)
np.shape(test_data_pca2)

(29, 29)

In [104]:
test_data_pca_df2 = pd.DataFrame(test_data_pca2)
test_data_pca_df2.shape

(29, 29)

In [0]:
test_ratio_standard_scaled_values2 = standard_scaler.fit_transform(test_data_pca_df2.values)
test_data_scaled2=pd.DataFrame(test_ratio_standard_scaled_values2)

In [107]:
clf_predict = clf.predict(test_data_scaled2) 
clf_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 2, 2, 0])

In [112]:
test_target_decoded_m

,0
0,11
1,1
2,7
3,12
4,9
5,18
6,3
7,14
8,20
9,15
